In [ ]:
import selenium
from selenium import webdriver
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import re

from sklearn.feature_extraction.text import TfidfVectorizer
from konlpy.tag import Kkma
from konlpy.utils import pprint

import threading
import time

import time
import datetime
from datetime import timedelta
import schedule

import pymysql
from selenium.webdriver.chrome.options import Options

In [ ]:
def start_crawling_seoul():
    # 페이지 접근 
    url ='https://safecity.seoul.go.kr/acdnt/roadIndex.do' # 사건사고
    driver.get(url)

    # 코드 추출
    html = driver.page_source
    soup = BeautifulSoup(html,'html.parser')

    # 5분내에 발생한 교통 이슈 크롤링 및 db저장
    for n_t in range(0,5):
        road_list = soup.find_all('div',{"class":"list_accident"})[n_t]
        road_list_info = road_list.find('a').get_text()
        road_list_st_time = re.sub("발생시간 : " , '', road_list.find('span',{"class":"add_txt02"}).get_text())
        road_list_end_time = re.sub("처리예상시간 : " , '',road_list.find('div',{"class":"list_acc_rit"}).get_text())

        if int((datetime.datetime.now() - datetime.datetime.strptime(road_list_st_time,"%Y.%m.%d %H:%M")).seconds/60) <= 5:
            sql = """ INSERT INTO cctv_road(road_info, road_st_time, road_end_time) VALUES('"""+str(road_list_info)+"""',
            '"""+str(road_list_st_time)+"""',
            '"""+str(road_list_end_time)+"""');"""
            news_db.execute(sql)
        db.commit()
        print(road_list)

In [ ]:
if __name__ == '__main__':

    # 데이터 베이스 연결
    db = pymysql.connect(host='database-big-a.cljkqcsbb9ok.ap-northeast-2.rds.amazonaws.com',port=3306, user='edu',passwd='multi1234!', db='iseeyou',charset='utf8')
    news_db = db.cursor()
    news_db

    # webdriver 객체 생성
    options = Options()
    options.add_argument("--headless")
    options.add_argument("window-size=1400,1500")
    driver= webdriver.Chrome(options=options)
    
    crawling = True
  # 5분마다 크롤링
    schedule.every(5).minutes.do(start_crawling_seoul)


    while crawling:
        schedule.run_pending()
        time.sleep(1)
    